In [2]:
import numpy as np
import pandas as pd
import requests
import xml.etree.ElementTree as ET
import re

import pprint

from Bio import Entrez

In [3]:
with open("../API_ignore.txt", "r") as f:
    lines = f.read()

entrez_api_key = lines.split(":")[1].strip()

In [4]:
afib_query = "https://eutils.ncbi.nlm.nih.gov/entrez/eutils/esearch.fcgi?db=pmc&term=atrial+fibrillation+AND+Review[ptyp]"

In [5]:
r = requests.get(afib_query)

In [6]:
r.content

b'<?xml version="1.0" encoding="UTF-8" ?>\n<!DOCTYPE eSearchResult PUBLIC "-//NLM//DTD esearch 20060628//EN" "https://eutils.ncbi.nlm.nih.gov/eutils/dtd/20060628/esearch.dtd">\n<eSearchResult><Count>40772</Count><RetMax>20</RetMax><RetStart>0</RetStart><IdList>\n<Id>6093735</Id>\n<Id>6092983</Id>\n<Id>6092761</Id>\n<Id>6091777</Id>\n<Id>6090232</Id>\n<Id>6090206</Id>\n<Id>6090074</Id>\n<Id>6090003</Id>\n<Id>6090002</Id>\n<Id>6090001</Id>\n<Id>6089703</Id>\n<Id>6089474</Id>\n<Id>6089464</Id>\n<Id>6089463</Id>\n<Id>6089132</Id>\n<Id>6089131</Id>\n<Id>6089130</Id>\n<Id>6089127</Id>\n<Id>6089122</Id>\n<Id>6088759</Id>\n</IdList><TranslationSet><Translation>     <From>atrial fibrillation</From>     <To>"atrial fibrillation"[MeSH Terms] OR ("atrial"[All Fields] AND "fibrillation"[All Fields]) OR "atrial fibrillation"[All Fields]</To>    </Translation><Translation>     <From>Review[All Fields]</From>     <To>"review"[All Fields] OR "review literature as topic"[MeSH Terms] OR "review"[All Fiel

In [4]:
entrez_query = "https://eutils.ncbi.nlm.nih.gov/entrez/eutils/efetch.fcgi?db=pmc&id={}&api_key={}"

In [5]:
def search(query):
    Entrez.email = 'jl56923@gmail.com'
    handle = Entrez.esearch(db='pubmed', 
                            sort='relevance', 
                            retmax='20',
                            retmode='xml', 
                            term=query)
    results = Entrez.read(handle)
    return results

In [6]:
search("atrial fibrillation")

DictElement({'Count': '71465', 'RetMax': '20', 'RetStart': '0', 'IdList': ['28543401', '29141842', '29729262', '29141778', '27888051', '28400367', '29485964', '29126922', '28886992', '28677250', '29052435', '29237510', '28823813', '28822701', '28807511', '28776267', '29223550', '28833764', '29126415', '28974629'], 'TranslationSet': [DictElement({'From': 'atrial fibrillation', 'To': '"atrial fibrillation"[MeSH Terms] OR ("atrial"[All Fields] AND "fibrillation"[All Fields]) OR "atrial fibrillation"[All Fields]'}, attributes={})], 'TranslationStack': [DictElement({'Term': '"atrial fibrillation"[MeSH Terms]', 'Field': 'MeSH Terms', 'Count': '47322', 'Explode': 'Y'}, attributes={}), DictElement({'Term': '"atrial"[All Fields]', 'Field': 'All Fields', 'Count': '163236', 'Explode': 'N'}, attributes={}), DictElement({'Term': '"fibrillation"[All Fields]', 'Field': 'All Fields', 'Count': '97627', 'Explode': 'N'}, attributes={}), 'AND', 'GROUP', 'OR', DictElement({'Term': '"atrial fibrillation"[Al

In [ ]:
Entrez.esearch()

In [10]:
Entrez.email = 'jl56923@gmail.com'
handle = Entrez.esearch(db='pubmed', sort='relevance', retmax='20', retmode='xml', term="atrial fibrillation")

## Step 1

First, we use esearch to send a query for all reviews & systematic reviews that have free full text for a specific topic; we want to get the PMIDs of these papers.

In [108]:
esearch_base_query = "https://eutils.ncbi.nlm.nih.gov/entrez/eutils/esearch.fcgi?"
    
review_pmids_query_dict = {
    "db": "pubmed",
    "sort": "relevance",
    "retmax": '10',
    "term": "{}+AND+((Review[ptyp]+OR+systematic[sb])+AND+free+full+text[sb])",
    "api_key": entrez_api_key
}

In [109]:
joined_terms = [k+"="+v for k, v in review_pmids_query_dict.items()] 

In [113]:
get_review_pmids_query = esearch_base_query + "&".join(joined_terms)

In [104]:
review_pmids_query_dict

{'sort': 'relevance',
 'retmax': 100,
 'term': '{}+AND+((Review[ptyp]+OR+systematic[sb])+AND+free+full+text[sb])',
 'api_key': 'b0b12c603fda132e7f526bd128008cf75a08'}

In [23]:
#"https://eutils.ncbi.nlm.nih.gov/entrez/eutils/esearch.fcgi?db=pubmed&term=asthma&Review[ptyp]"
# https://eutils.ncbi.nlm.nih.gov/entrez/eutils/esearch.fcgi?db=pubmed&sort=relevance&retmax=100&term=atrial+fibrillation+AND+((Review[ptyp]+OR+systematic[sb])+AND+free+full+text[sb])

get_review_pmids_query = "https://eutils.ncbi.nlm.nih.gov/entrez/eutils/esearch.fcgi?db=pubmed&term={}+AND+((Review[ptyp]+OR+systematic[sb])+AND+free+full+text[sb])&api_key={}"

In [114]:
first_search = "atrial+fibrillation"

In [115]:
# Once we send this query to pubmed using eutils, we get back an xml object which we can store in a tree.
r = requests.get(get_review_pmids_query.format(first_search, entrez_api_key))

In [116]:
tree = ET.ElementTree(ET.fromstring(r.content))
root = tree.getroot()

pmids = root.findall('.//Id')

pmid_list = [pmid.text for pmid in pmids]

In [117]:
pmid_list

['28617620',
 '28272807',
 '27681890',
 '29298352',
 '29073191',
 '28822529',
 '27802324',
 '27338593',
 '27613773',
 '26464292',
 '28774317',
 '28264715',
 '28229164',
 '28695554',
 '28273368',
 '28194602',
 '28750152',
 '28143406',
 '25701017',
 '27342651',
 '27984303',
 '27920399',
 '28381269',
 '27599725',
 '28450367',
 '28484965',
 '26391136',
 '28077320',
 '27368043',
 '28502818',
 '28416818',
 '28720644',
 '27663299',
 '26940476',
 '26791994',
 '27551927',
 '26306128',
 '26306124',
 '27811208',
 '25806470',
 '27815538',
 '27082929',
 '26412599',
 '26849763',
 '28076631',
 '27451924',
 '28962617',
 '29518134',
 '29183961',
 '25262686',
 '27492815',
 '25858534',
 '28293114',
 '27797891',
 '28514878',
 '27502864',
 '28150281',
 '25425976',
 '26149044',
 '28992764',
 '26152444',
 '25807223',
 '26306127',
 '26306125',
 '25071357',
 '26763380',
 '26149040',
 '28471498',
 '27389440',
 '25773559',
 '26786546',
 '28956288',
 '26604772',
 '30050737',
 '24854873',
 '25389649',
 '26561858',

## Step 2

Now that we have the PMIDs for the 20 review papers returned by esearch, we have to convert the PMIDs into PMCIDs. In order to convert the PMIDs to PMCIDs, we have to use the ID converter provided by the NCBI, as outlined here: https://www.ncbi.nlm.nih.gov/pmc/tools/id-converter-api/

In [118]:
# Let's convert each PMID into a PMCID. The JSON that is returned from this request always has a key 'records'.
# Check the dictionary inside of 'records'; if there is a key called 'errmsg', then you know that
# the convert request failed. Otherwise, check to see if the dictionary inside of records has a key called 
# 'pmcid'. If it does, grab the value of the key 'pmcid' and store it. We'll use that PMCID to query PMC to
# fetch the xml of the full paper.

convert_PMID_query = "https://www.ncbi.nlm.nih.gov/pmc/utils/idconv/v1.0/?tool=review_assistant&email=jl56923@gmail.com&ids={}&format=json"

In [119]:
pmcid_list = []

for pmid in pmid_list:
    r = requests.get(convert_PMID_query.format(pmid))
    result = r.json()
    records_dict = result['records'][0]
    if 'errmsg' in records_dict:
        pass
    else:
        if 'pmcid' in records_dict:
            pmcid_list.append(records_dict['pmcid'])

In [120]:
pmcid_list

['PMC5598874',
 'PMC5752005',
 'PMC5658096',
 'PMC5560908',
 'PMC5089715',
 'PMC5071280',
 'PMC5079045',
 'PMC4952027',
 'PMC5543536',
 'PMC5340010',
 'PMC5442605',
 'PMC5380695',
 'PMC5286679',
 'PMC4766963',
 'PMC5942796',
 'PMC5321114',
 'PMC5382469',
 'PMC5500874',
 'PMC5465041',
 'PMC4599513',
 'PMC5346472',
 'PMC4937957',
 'PMC5585859',
 'PMC5586302',
 'PMC4819630',
 'PMC5046840',
 'PMC4547665',
 'PMC4547682',
 'PMC5427484',
 'PMC5622555',
 'PMC5843263',
 'PMC5704695',
 'PMC5122472',
 'PMC5345987',
 'PMC5108192',
 'PMC4189345',
 'PMC5726608',
 'PMC4472367',
 'PMC4547664',
 'PMC4547683',
 'PMC4110594',
 'PMC4732179',
 'PMC5482349',
 'PMC5656712',
 'PMC4630199',
 'PMC6059525',
 'PMC4764082',
 'PMC4642960',
 'PMC4788372',
 'PMC5588987',
 'PMC5933600',
 'PMC4724415',
 'PMC4246362',
 'PMC4814009',
 'PMC5457732',
 'PMC5487882',
 'PMC5382449',
 'PMC5403606',
 'PMC4329775',
 'PMC4957677',
 'PMC4051329',
 'PMC4731871',
 'PMC4547684',
 'PMC5234257',
 'PMC6009792',
 'PMC5331111']

In [138]:
len(pmcid_list)

66

In [57]:
first_PMID = "30104821"

In [60]:
second_PMID = "30103676"

In [61]:
r = requests.get(convert_PMID.format(second_PMID))

In [63]:
result = r.json()

In [70]:
result

{'status': 'ok',
 'responseDate': '2018-08-17 14:31:18',
 'request': 'tool=my_tool;email=my_email%40example.com;ids=30103676;format=json',
 'records': [{'pmid': '30103676',
   'live': 'false',
   'status': 'error',
   'errmsg': 'invalid article id'}]}

In [83]:
'errmsg' in result['records'][0]

True

In [71]:
r2 = requests.get(convert_PMID.format(first_PMID))

In [72]:
result2 = r2.json()

In [77]:
result2

{'status': 'ok',
 'responseDate': '2018-08-17 14:35:22',
 'request': 'tool=my_tool;email=my_email%40example.com;ids=30104821;format=json',
 'records': [{'pmcid': 'PMC6066889',
   'pmid': '30104821',
   'doi': '10.4103/joacp.JOACP_352_16',
   'versions': [{'pmcid': 'PMC6066889.1', 'current': 'true'}]}]}

In [84]:
'errmsg' in result2['records'][0]

False

In [54]:
tree = ET.ElementTree(ET.fromstring(r.content))
root = tree.getroot()

In [55]:
ET.dump(tree)

<pmcids status="ok">
<request idtype="pmid" pmids="" showaiid="no" versions="yes">
<echo>tool=my_tool;email=my_email%40example.com;ids=30104821</echo>
</request>
<record doi="10.4103/joacp.JOACP_352_16" pmcid="PMC6066889" pmid="30104821" requested-id="30104821"><versions><version current="true" pmcid="PMC6066889.1" /></versions></record>
</pmcids>


## Step 3

Now that we have the list of pmcids, we can use efetch to get the xml of these papers.

In [93]:
get_pmc_xml_query = "https://eutils.ncbi.nlm.nih.gov/entrez/eutils/efetch.fcgi?db=pmc&id={}"
get_pmc_xml_query2 = "https://eutils.ncbi.nlm.nih.gov/entrez/eutils/efetch.fcgi?db=pmc&id={}tool=review_assistant&email=jl56923@gmail.com"

In [121]:
pmcid_list[0]

'PMC5598874'

In [126]:
first_pmcid = pmcid_list[1]

In [127]:
r = requests.get(get_pmc_xml_query.format(first_pmcid))

In [128]:
tree = ET.ElementTree(ET.fromstring(r.content))
root = tree.getroot()

In [189]:
title = root.find('.//article-title')

In [192]:
title.text

'The safety and efficacy of hybrid ablation for the treatment of atrial fibrillation: A meta-analysis'

In [205]:
# Let's grab all the nodes that have the tag 'sec'
sections = root.findall('.//sec')

In [215]:
ET.dump(sections[4])

<sec id="sec005" sec-type="intro">
      <title>Introduction</title>
      <p>Atrial fibrillation (AF) is the most common type of arrhythmia reported in clinical practice. As this condition is associated with an increased risk of major complications, such as thrombosis and stroke, finding optimal therapies for AF has long been a topic of interest among clinicians. Although the Cox Maze III procedure has been considered the gold standard for the surgical treatment of AF since 1991, [<xref ref-type="bibr" rid="pone.0190170.ref001">1</xref>] it has not been used extensively due to its complexity and invasive character. Pulmonary vein isolation (PVI), the standard catheter ablation procedure for paroxysmal AF (PrAF), has success rates exceeding 70% and acceptable complication rates [<xref ref-type="bibr" rid="pone.0190170.ref002">2</xref>]. However, neither epicardial ablation nor endocardial ablation alone can achieve a satisfactory result in patients with long-standing persistent AF (LSP

In [223]:
sections[5].attrib

{'sec-type': 'materials|methods', 'id': 'sec006'}

In [187]:
ET.dump(tree)

<pmc-articleset xmlns:ns0="http://www.w3.org/1999/xlink"><article article-type="research-article">
  
  <front>
    <journal-meta>
      <journal-id journal-id-type="nlm-ta">PLoS One</journal-id>
      <journal-id journal-id-type="iso-abbrev">PLoS ONE</journal-id>
      <journal-id journal-id-type="publisher-id">plos</journal-id>
      <journal-id journal-id-type="pmc">plosone</journal-id>
      <journal-title-group>
        <journal-title>PLoS ONE</journal-title>
      </journal-title-group>
      <issn pub-type="epub">1932-6203</issn>
      <publisher>
        <publisher-name>Public Library of Science</publisher-name>
        <publisher-loc>San Francisco, CA USA</publisher-loc>
      </publisher>
    </journal-meta>
    <article-meta>
      <article-id pub-id-type="pmid">29298352</article-id>
      <article-id pub-id-type="pmc">5752005</article-id>
      <article-id pub-id-type="doi">10.1371/journal.pone.0190170</article-id>
      <article-id pub-id-type="publisher-id">PONE-D-17-2939

                  <td align="center" colspan="1" rowspan="1">Bulava(2015)</td>
                  <td align="center" colspan="1" rowspan="1">50</td>
                  <td align="center" colspan="1" rowspan="1">LSP</td>
                  <td align="center" colspan="1" rowspan="1">3.5</td>
                  <td align="center" colspan="1" rowspan="1">50</td>
                  <td align="center" colspan="1" rowspan="1">100</td>
                  <td align="center" colspan="1" rowspan="1">0</td>
                  <td align="center" colspan="1" rowspan="1">-</td>
                  <td align="center" colspan="1" rowspan="1">0</td>
                  <td align="center" colspan="1" rowspan="1">-</td>
                  <td align="center" colspan="1" rowspan="1">0</td>
                </tr>
                <tr>
                  <td align="center" colspan="1" rowspan="1">12</td>
                  <td align="center" colspan="1" rowspan="1">de Asmundis(2016)</td>
                  <td align="center" 

                  <td align="center" colspan="1" rowspan="1">R-VATS</td>
                  <td align="center" colspan="1" rowspan="1">Staged</td>
                  <td align="center" colspan="1" rowspan="1">Both</td>
                  <td align="center" colspan="1" rowspan="1">5</td>
                  <td align="center" colspan="1" rowspan="1">48</td>
                  <td align="center" colspan="1" rowspan="1">5</td>
                </tr>
              </tbody>
            </table>
          </alternatives>
          <table-wrap-foot>
            <fn id="t002fn001">
              <p>Nr = number; Repeat CA = repeat catheter ablation; LAA = left atrial appendage; VATS = video-assisted thoracoscopy; R = right side; L = left-side; LAP = laparoscope; NOS = Newcastle–Ottawa Scale.</p>
            </fn>
          </table-wrap-foot>
        </table-wrap>
      </sec>
      <sec id="sec012">
        <title>Overall meta-analysis</title>
        <sec id="sec013">
          <title>Efficacy</title

        <mixed-citation publication-type="journal"><name><surname>Zembala</surname><given-names>M</given-names></name>, <name><surname>Filipiak</surname><given-names>K</given-names></name>, <name><surname>Kowalski</surname><given-names>O</given-names></name>, <name><surname>Buchta</surname><given-names>P</given-names></name>, <name><surname>Niklewski</surname><given-names>T</given-names></name>, <name><surname>Nadziakiewicz</surname><given-names>P</given-names></name>, <etal>et al</etal>
<article-title>Staged hybrid ablation for persistent and longstanding persistent atrial fibrillation effectively restores sinus rhythm in long-term observation</article-title>. <source>Arch Med Sci</source>. <year>2017</year>;<volume>13</volume>(<issue>1</issue>): <fpage>109</fpage>–<lpage>117</lpage>. <comment>doi: <ext-link ext-link-type="uri" ns0:href="https://doi.org/10.5114/aoms.2015.53960">10.5114/aoms.2015.53960</ext-link></comment>
<pub-id pub-id-type="pmid">28144262</pub-id></mixed-citation>
 

In [188]:
tree.write("test.xml")

In [182]:
for child in root:
    for second_child in child:
        print(second_child.tag, second_child.attrib)
        for third_child in second_child:
            print(third_child.tag, third_child.attrib)
        print("\n")

front {}
journal-meta {}
article-meta {}
notes {}


body {}
sec {'sec-type': 'intro', 'id': 'sec005'}
sec {'sec-type': 'materials|methods', 'id': 'sec006'}
sec {'sec-type': 'results', 'id': 'sec010'}
sec {'sec-type': 'conclusions', 'id': 'sec021'}
sec {'id': 'sec027'}
sec {'sec-type': 'conclusions', 'id': 'sec028'}
sec {'sec-type': 'supplementary-material', 'id': 'sec029'}


back {}
ack {}
ref-list {}




In [168]:
root[0][1].tag

'body'

In [150]:
import xml_to_dict as xml_to_dict

In [158]:
xmldict = xml_to_dict.XmlDictConfig(root[0])

In [160]:
xmldict.keys()

dict_keys(['article-type', 'front', 'body', 'back'])

In [167]:
xmldict['body']

{'sec': [{'sec-type': 'intro',
   'id': 'sec005',
   'title': 'Introduction',
   'p': {'xref': ['1', '2', '3', '4']}},
  {'sec-type': 'materials|methods',
   'id': 'sec006',
   'title': 'Methods',
   'sec': {'id': 'sec009',
    'title': 'Data analysis',
    'p': {'xref': {'rid': 'pone.0190170.ref007', 'ref-type': 'bibr'},
     'sup': '2'}}},
  {'sec-type': 'results',
   'id': 'sec010',
   'title': 'Results',
   'sec': {'id': 'sec020',
    'title': 'Sensitivity analysis',
    'p': {'sup': '2'}}},
  {'sec-type': 'conclusions',
   'id': 'sec021',
   'title': 'Discussion',
   'sec': {'id': 'sec026',
    'title': 'Endpoints for ablation',
    'p': {'xref': {'rid': 'pone.0190170.ref028', 'ref-type': 'bibr'},
     'italic': 'P'}}},
  {'id': 'sec027',
   'title': 'Limitations',
   'p': 'Due to the novelty of this procedure, the existing studies discussing hybrid ablation are mostly single-arm trials without control groups. Thus, there is no reliable comparison between the hybrid procedure and 

In [185]:
xmldict2 = xml_to_dict.XmlDictConfig(root[0])

In order to get to the minimal viable product going, let's just concentrate on extracting the bare text from the xml files of the first 5 review papers, and then saving them as txt files. In the next notebook, extractive summarization, I'll implement a very simple extractive summarizer on all of the documents together. Once that summarizer is working well, I'll figure out how to cluster paragraphs, and then run the summarizer on each of those clusters.

In [249]:
for i in range(5):
    r = requests.get(get_pmc_xml_query.format(pmcid_list[i]))

    tree = ET.ElementTree(ET.fromstring(r.content))
    root = tree.getroot()

    all_text = root.findall('.//p')

    raw_text = ""

    for texts in all_text:
        raw_text += " ".join(texts.itertext())
    
    filename = "af_paper" + str(i+1) + ".txt"
    with open("documents/"+filename, "w") as f:
        f.write(raw_text)

In [339]:
ET.dump(tree)

<pmc-articleset xmlns:ns0="http://www.w3.org/1999/xlink"><article article-type="research-article">
  
  <front>
    <journal-meta>
      <journal-id journal-id-type="nlm-ta">PLoS One</journal-id>
      <journal-id journal-id-type="iso-abbrev">PLoS ONE</journal-id>
      <journal-id journal-id-type="publisher-id">plos</journal-id>
      <journal-id journal-id-type="pmc">plosone</journal-id>
      <journal-title-group>
        <journal-title>PLoS ONE</journal-title>
      </journal-title-group>
      <issn pub-type="epub">1932-6203</issn>
      <publisher>
        <publisher-name>Public Library of Science</publisher-name>
        <publisher-loc>San Francisco, CA USA</publisher-loc>
      </publisher>
    </journal-meta>
    <article-meta>
      <article-id pub-id-type="pmid">27802324</article-id>
      <article-id pub-id-type="pmc">5089715</article-id>
      <article-id pub-id-type="doi">10.1371/journal.pone.0165790</article-id>
      <article-id pub-id-type="publisher-id">PONE-D-16-1656

                  <td align="center" colspan="1" rowspan="1">+/-</td>
                  <td align="center" colspan="1" rowspan="1">+</td>
                  <td align="center" colspan="1" rowspan="1">+/-</td>
                  <td align="center" colspan="1" rowspan="1">+</td>
                </tr>
                <tr>
                  <td align="left" colspan="1" rowspan="1">Measurement error</td>
                  <td align="center" colspan="1" rowspan="1">n/r</td>
                  <td align="center" colspan="1" rowspan="1">n/r</td>
                  <td align="center" colspan="1" rowspan="1">n/r</td>
                  <td align="center" colspan="1" rowspan="1">n/r</td>
                  <td align="center" colspan="1" rowspan="1">n/r</td>
                </tr>
                <tr>
                  <td align="left" colspan="1" rowspan="1">Content Validity</td>
                  <td align="center" colspan="1" rowspan="1">?</td>
                  <td align="center" colspan="1" rowspan=

        <mixed-citation publication-type="journal"><name><surname>de Vries</surname><given-names>M</given-names></name>, <name><surname>Ouwendijk</surname><given-names>R</given-names></name>, <name><surname>Kessels</surname><given-names>AG</given-names></name>, <name><surname>de Haan</surname><given-names>MW</given-names></name>, <name><surname>Flobbe</surname><given-names>K</given-names></name>, <name><surname>Hunink</surname><given-names>MG</given-names></name>, <etal>et al</etal>
<article-title>Comparison of generic and disease-specific questionnaires for the assessment of quality of life in patients with peripheral arterial disease</article-title>. <source>J Vasc Surg</source>. <year>2005</year>;<volume>41</volume>(<issue>2</issue>):<fpage>261</fpage>–<lpage>8</lpage>. <comment>doi: <ext-link ext-link-type="uri" ns0:href="http://dx.doi.org/10.1016/j.jvs.2004.11.022">10.1016/j.jvs.2004.11.022</ext-link></comment>
<pub-id pub-id-type="pmid">15768008</pub-id></mixed-citation>
      </

        <mixed-citation publication-type="journal"><name><surname>Dobson</surname><given-names>F</given-names></name>, <name><surname>Hinman</surname><given-names>RS</given-names></name>, <name><surname>Hall</surname><given-names>M</given-names></name>, <name><surname>Terwee</surname><given-names>CB</given-names></name>, <name><surname>Roos</surname><given-names>EM</given-names></name>, <name><surname>Bennell</surname><given-names>KL</given-names></name>. <article-title>Measurement properties of performance-based measures to assess physical function in hip and knee osteoarthritis: a systematic review</article-title>. <source>Osteoarthritis Cartilage</source>. <year>2012</year>;<volume>20</volume>(<issue>12</issue>):<fpage>1548</fpage>–<lpage>62</lpage>. <comment>doi: <ext-link ext-link-type="uri" ns0:href="http://dx.doi.org/10.1016/j.joca.2012.08.015">10.1016/j.joca.2012.08.015</ext-link></comment>
<pub-id pub-id-type="pmid">22944525</pub-id></mixed-citation>
      </ref>
      <ref id

In [340]:
root = tree.getroot()

In [341]:
sections = root.findall(".//sec")

In [344]:
for section in sections:
    print(section.tag)
    print(section.attrib)
    print(list(section))
    section_title = section.find(".//title")
    print(section_title.text)
    print("----\n")

sec
{'id': 'sec001'}
[<Element 'title' at 0x10c75ccc8>, <Element 'p' at 0x10c75cc28>]
Background
----

sec
{'id': 'sec002'}
[<Element 'title' at 0x10c75c408>, <Element 'p' at 0x10c75cbd8>]
Methods and Results
----

sec
{'id': 'sec003'}
[<Element 'title' at 0x10c75cef8>, <Element 'p' at 0x10c75c9a8>]
Interpretation
----

sec
{'sec-type': 'intro', 'id': 'sec004'}
[<Element 'title' at 0x10c91c598>, <Element 'p' at 0x10c91c9f8>, <Element 'p' at 0x10c6ef3b8>, <Element 'p' at 0x10c6ef458>, <Element 'p' at 0x10c6ef228>]
Introduction
----

sec
{'sec-type': 'materials|methods', 'id': 'sec005'}
[<Element 'title' at 0x10c6efbd8>, <Element 'sec' at 0x10c6ef638>, <Element 'sec' at 0x10c6efae8>]
Methods
----

sec
{'id': 'sec006'}
[<Element 'title' at 0x10c6efef8>, <Element 'p' at 0x10c6ef598>, <Element 'p' at 0x10c6ef908>, <Element 'p' at 0x10c6ef4f8>]
Eligibility criteria and search strategy
----

sec
{'id': 'sec007'}
[<Element 'title' at 0x10c6efb38>, <Element 'p' at 0x10c6ef1d8>, <Element 'p' at 

In [347]:
for section in sections[:17]:
    print(section.tag)
    print(section.attrib)
    print(list(section))
    section_title = section.find(".//title")
    print(section_title.text)
    print("----\n")

sec
{'id': 'sec001'}
[<Element 'title' at 0x10c75ccc8>, <Element 'p' at 0x10c75cc28>]
Background
----

sec
{'id': 'sec002'}
[<Element 'title' at 0x10c75c408>, <Element 'p' at 0x10c75cbd8>]
Methods and Results
----

sec
{'id': 'sec003'}
[<Element 'title' at 0x10c75cef8>, <Element 'p' at 0x10c75c9a8>]
Interpretation
----

sec
{'sec-type': 'intro', 'id': 'sec004'}
[<Element 'title' at 0x10c91c598>, <Element 'p' at 0x10c91c9f8>, <Element 'p' at 0x10c6ef3b8>, <Element 'p' at 0x10c6ef458>, <Element 'p' at 0x10c6ef228>]
Introduction
----

sec
{'sec-type': 'materials|methods', 'id': 'sec005'}
[<Element 'title' at 0x10c6efbd8>, <Element 'sec' at 0x10c6ef638>, <Element 'sec' at 0x10c6efae8>]
Methods
----

sec
{'id': 'sec006'}
[<Element 'title' at 0x10c6efef8>, <Element 'p' at 0x10c6ef598>, <Element 'p' at 0x10c6ef908>, <Element 'p' at 0x10c6ef4f8>]
Eligibility criteria and search strategy
----

sec
{'id': 'sec007'}
[<Element 'title' at 0x10c6efb38>, <Element 'p' at 0x10c6ef1d8>, <Element 'p' at 

I tried using regex to just pull the xml tags out of what gets returned by r.content, but this was too messy to work with. Instead, the code below looks for an 'abstract' section in the xml and extracts the text there to get the abstract, and then it looks for the 'body' section and extracts the text there.

In [277]:
i = 0

In [278]:
r = requests.get(get_pmc_xml_query.format(pmcid_list[i]))

tree = ET.ElementTree(ET.fromstring(r.content))
root = tree.getroot()

In [279]:
abstract = root.find(".//abstract")

In [282]:
ET.dump(abstract)

<abstract>
        <sec>
          <title>Background</title>
          <p>In contemporary atrial fibrillation trials most deaths are cardiac related, whereas stroke and bleeding represent only a small subset of deaths. We aimed to evaluate the long-term risk of cardiac events and all-cause mortality in individuals with atrial fibrillation compared to no atrial fibrillation.</p>
        </sec>
        <sec>
          <title>Design</title>
          <p>A systematic review and meta-analysis of studies published between 1 January 2006 and 21 October 2016.</p>
        </sec>
        <sec>
          <title>Methods</title>
          <p>Four databases were searched. Studies had follow-up of at least 500 stable patients for either cardiac endpoints or all-cause mortality for 12 months or longer. Publication bias was evaluated and random effects models were used to synthesise the results. Heterogeneity between studies was examined by subgroup and meta-regression analyses.</p>
        </sec>
    

In [286]:
abstract_paragraphs = abstract.findall(".//p")

In [293]:
abstract_text = ""

for paragraph in abstract_paragraphs:
        abstract_text += " ".join(paragraph.itertext())
        abstract_text += " "
        print(" ".join(paragraph.itertext()))
        print("-----\n")

abstract_text = abstract_text.strip()

In contemporary atrial fibrillation trials most deaths are cardiac related, whereas stroke and bleeding represent only a small subset of deaths. We aimed to evaluate the long-term risk of cardiac events and all-cause mortality in individuals with atrial fibrillation compared to no atrial fibrillation.
-----

A systematic review and meta-analysis of studies published between 1 January 2006 and 21 October 2016.
-----

Four databases were searched. Studies had follow-up of at least 500 stable patients for either cardiac endpoints or all-cause mortality for 12 months or longer. Publication bias was evaluated and random effects models were used to synthesise the results. Heterogeneity between studies was examined by subgroup and meta-regression analyses.
-----

A total of 15 cohort studies was included. Analyses indicated that atrial fibrillation was associated with an increased risk of myocardial infarction (relative risk (RR) 1.54, 95% confidence interval (CI) 1.26–1.85), all-cause mortal

In [294]:
abstract_text

'In contemporary atrial fibrillation trials most deaths are cardiac related, whereas stroke and bleeding represent only a small subset of deaths. We aimed to evaluate the long-term risk of cardiac events and all-cause mortality in individuals with atrial fibrillation compared to no atrial fibrillation. A systematic review and meta-analysis of studies published between 1 January 2006 and 21 October 2016. Four databases were searched. Studies had follow-up of at least 500 stable patients for either cardiac endpoints or all-cause mortality for 12 months or longer. Publication bias was evaluated and random effects models were used to synthesise the results. Heterogeneity between studies was examined by subgroup and meta-regression analyses. A total of 15 cohort studies was included. Analyses indicated that atrial fibrillation was associated with an increased risk of myocardial infarction (relative risk (RR) 1.54, 95% confidence interval (CI) 1.26–1.85), all-cause mortality (RR 1.95, 95% CI

In [283]:
body = root.find(".//body")

In [295]:
body_paragraphs = body.findall(".//p")

body_text = ""

for paragraph in body_paragraphs:
        body_text += " ".join(paragraph.itertext())
        body_text += " "
        
body_text = body_text.strip()

In [296]:
body_text

'Current treatment of thromboembolic complications associated with atrial fibrillation (AF) has mainly focused on the prevention of stroke. 1 , 2  Notwithstanding, patients with AF frequently develop coronary heart disease (CHD). Similar cardiovascular risk factors for CHD and AF have been suggested to reflect a common pathway of underlying vascular disease. 3 In a recent meta-analysis of patients with AF, cardiac deaths accounted for 46% of all deaths during follow-up, whereas non-haemorrhagic stroke/systemic embolism and haemorrhage-related deaths represented only 5.7% and 5.6% of the total mortality, respectively. 4  As part of the Atherosclerosis Risk in Communities (ARIC) study, 5  including participants free of CHD at baseline, AF was associated with a 63% increased risk of incident myocardial infarction (MI). These studies limited inclusion to either AF-only or CHD-free patients. The objective of the present systematic review was therefore to summarise the evidence from contempo

In [285]:
ET.dump(body)

<body xmlns:ns0="http://www.w3.org/1999/xlink">
    <sec id="sec1-2047487317715769" sec-type="intro">
      <title>Introduction</title>
      <p>Current treatment of thromboembolic complications associated with atrial fibrillation (AF) has mainly focused on the prevention of stroke.<sup><xref ref-type="bibr" rid="bibr1-2047487317715769">1</xref>,<xref ref-type="bibr" rid="bibr2-2047487317715769">2</xref></sup> Notwithstanding, patients with AF frequently develop coronary heart disease (CHD). Similar cardiovascular risk factors for CHD and AF have been suggested to reflect a common pathway of underlying vascular disease.<sup><xref ref-type="bibr" rid="bibr3-2047487317715769">3</xref></sup></p>
      <p>In a recent meta-analysis of patients with AF, cardiac deaths accounted for 46% of all deaths during follow-up, whereas non-haemorrhagic stroke/systemic embolism and haemorrhage-related deaths represented only 5.7% and 5.6% of the total mortality, respectively.<sup><xref ref-type="bibr" r

</th><th colspan="1" rowspan="1">Summary RR (95% CI)<sup><xref ref-type="table-fn" rid="table-fn7-2047487317715769">a</xref></sup></th><th colspan="1" rowspan="1">RR=1 (z)</th><th colspan="1" rowspan="1"><italic>P</italic> value</th><th colspan="1" rowspan="1">I<sup><xref ref-type="bibr" rid="bibr2-2047487317715769">2</xref></sup> (%)</th></tr></thead><tbody align="left" valign="top"><tr><td colspan="1" rowspan="1">Endpoint myocardial infarction</td><td colspan="1" rowspan="1" /><td colspan="1" rowspan="1" /><td colspan="1" rowspan="1" /><td colspan="1" rowspan="1" /><td colspan="1" rowspan="1" /></tr><tr><td colspan="1" rowspan="1"> All studies</td><td colspan="1" rowspan="1">16</td><td colspan="1" rowspan="1">1.54 (1.28–1.85)</td><td colspan="1" rowspan="1">4.64</td><td colspan="1" rowspan="1">&lt;0.001</td><td colspan="1" rowspan="1">80.9</td></tr><tr><td colspan="1" rowspan="1">Subdivision</td><td colspan="1" rowspan="1" /><td colspan="1" rowspan="1" /><td colspan="1" rowspan="1" /

Now let's try the second AF paper.

In [325]:
i = 3

In [326]:
r = requests.get(get_pmc_xml_query.format(pmcid_list[i]))

tree = ET.ElementTree(ET.fromstring(r.content))
root = tree.getroot()

In [327]:
abstract = root.find(".//abstract")

In [328]:
ET.dump(abstract)

<abstract id="abs0005">
        <p>Atrial fibrillation is the most common arrhythmia worldwide with increasing frequency noted with age. Hyperthyroidism is a well-known cause of atrial fibrillation with a 16%–60% prevalence of atrial fibrillation in patients with known hyperthyroidism Ross et al. (2016). While hyperthyroidism as a causative factor of atrial fibrillation is well established, this literature review aims to answer several questions on this topic including:</p>
        <p>1. The relationship of atrial fibrillation to hyperthyroidism</p>
        <p>2. Atrial fibrillation as a predictor of hyperthyroidism</p>
        <p>3. The pathophysiology of thyrotoxic atrial fibrillation</p>
        <p>4. Subclinical hyperthyroidism and the relationship with atrial fibrillation</p>
        <p>5. Cardioversion and Catheter ablation of hyperthyroid patients with atrial fibrillation</p>
        <p>6. Thrombotic risk of hyperthyroid patients with atrial fibrillation</p>
        <p>7. Manage

In [329]:
abstract_paragraphs = abstract.findall(".//p")

In [330]:
abstract_text = ""

for paragraph in abstract_paragraphs:
        abstract_text += " ".join(paragraph.itertext())
        abstract_text += " "
#         print(" ".join(paragraph.itertext()))
#         print("-----\n")

abstract_text = abstract_text.strip()

In [331]:
abstract_text

'Atrial fibrillation is the most common arrhythmia worldwide with increasing frequency noted with age. Hyperthyroidism is a well-known cause of atrial fibrillation with a 16%–60% prevalence of atrial fibrillation in patients with known hyperthyroidism Ross et al. (2016). While hyperthyroidism as a causative factor of atrial fibrillation is well established, this literature review aims to answer several questions on this topic including: 1. The relationship of atrial fibrillation to hyperthyroidism 2. Atrial fibrillation as a predictor of hyperthyroidism 3. The pathophysiology of thyrotoxic atrial fibrillation 4. Subclinical hyperthyroidism and the relationship with atrial fibrillation 5. Cardioversion and Catheter ablation of hyperthyroid patients with atrial fibrillation 6. Thrombotic risk of hyperthyroid patients with atrial fibrillation 7. Management of Thyrotoxic Atrial fibrillation 8. Pharmacological rhythm control in patients with hyperthyroidism and atrial fibrillation 9. Treatm

In [332]:
body = root.find(".//body")

In [333]:
body_paragraphs = body.findall(".//p")

body_text = ""

for paragraph in body_paragraphs:
        body_text += " ".join(paragraph.itertext())
        body_text += " "
        
body_text = body_text.strip()

In [334]:
body_text

'Hyperthyroidism, or thyrotoxicosis occurs due to excess release of thyroid hormone due to an overactive thyroid gland or passive release of the stored hormone. Additionally, hyperthyroidism occurs from over treatment with thyroid hormone. Hyperthyroidism is generally considered overt or subclinical, depending on the biochemical severity of the hyperthyroidism. Overt hyperthyroidism is defined as suppressed (usually undetectable) thyrotropin (TSH) and elevated levels of triiodothyronine (T3) and/or estimated free thyroxine (free T4). Subclinical hyperthyroidism is defined as a low or undetectable serum TSH with values within the normal reference range for both T3 and free T4. 1  Hyperthyroidism should be considered the potential illness whenever TSH level is subnormal. The prevalence of hyperthyroidism in the United States is approximately 1.2% (0.5% overt and 0.7% subclinical). 2  In an older community in Baltimore, the prevalence of low TSH was 9.6% for participants on thyroid hormon

Okay, so first finding the abstract and then the body nodes, and then going through each of those and joining together the paragraphs seems to work relatively well. We'll go ahead and write the abstract and body texts to files instead. We'll also define a function that can take an XML node, look for all the paragraphs, join them together and return a clean string.

In [335]:
def get_paragraphs_as_clean_string(xml_node):
    node_paragraphs = xml_node.findall(".//p")

    clean_string = ""

    for paragraph in node_paragraphs:
        clean_string += " ".join(paragraph.itertext())
        clean_string += " "
        
    clean_string = clean_string.strip()
    
    return clean_string

In [338]:
for i in range(5):
    r = requests.get(get_pmc_xml_query.format(pmcid_list[i]))

    tree = ET.ElementTree(ET.fromstring(r.content))
    root = tree.getroot()
    
    abstract = root.find(".//abstract")
    abstract_text = get_paragraphs_as_clean_string(abstract)
    
    with open(f"documents/af_paper{i+1}_abstract.txt", "w") as f:
        f.write(abstract_text)
    
    body = root.find(".//body")
    body_text = get_paragraphs_as_clean_string(body)
    
    # Let's also get rid of all of the references that are in square brackets, if there are any.
    body_text = re.sub(r' \[.*?]', "", body_text)
    
    with open(f"documents/af_paper{i+1}_body.txt", "w") as f:
        f.write(body_text)